# Functional Programming
Suppose you live in a world where you don't use classes (gasp!). Why in the world would you want to do that? The reason is that some problems are better suited when you don't mutate an underlying object. OOP uses attributes to not only keep state but mutate state. Because the statefulness of an object can be mutated, there are use cases where mutations hinders rather than helps: race conditions in threaded code, debugging complex mutations across multiple method calls, non-parallelizable so cannot distribute the computation, etc. Hence, some problems are better where you don't remember something about the object after it is used--it is stateless.

Functional Programming (often abbreviated as FP) treat objects as immutable and create new objects when needed--instead of mutating an object. An important nuance about FP is that it isn't about making things "stateless"; for example, there are still stateful things like variables and data structures (ie lists) in FP. FP is about preventing *mutating* state. In my opinion, FP tries to enforce a function as a single unit of transformation, making a function call atomic. Either a function succeeded in transforming the data or it didn't. You aren't supposed to inspect intermediate results--which is what you can do in OOP.

At a high level, I want to leave you with the idea that, in contrast to OOP, FP deliberately separates data and code. In FP, data is data and code (ie functions) is code. A procedural (or imperative) style tends to run data through code, store the result, run data through more code, etc--you are manipulating data. In constrast, a functional style tends to manipulate functions: construct more complex functions out of simpler functions (like Lego bricks) and then run data through the complex function.

If you are a boring, math professor who likes to think that CS is just applied math, then you probably think in terms of the textbook definitions of Functional Programming: a) **immutability**, b) **pure functions**, and **high-order functions**.
* **Immutability**: nothing ever changes. What you put into a function, you get something else out. Stay the way you are--never change. Don't update yourself; transform into something new.
* **Pure functions**: functions ideally should be completely deterministic in that given a specific input, the output will ALWAYS return the same output. In fact, nothing else should change at all; a __side effect__ is something that changes outside the scope of the function. Pure functions, by definition, cannot have a side effect. Sometimes, this constrant is too difficult, so the lesser constraint is `idempotency`: if something else changes (that is not the output), then something always changes in exactly same way--the side effect is deterministic.
* **Higher-order functions**: since functions are first-class citizens in Python, what they means is that you can pass functions into functions. You can have a function return a function. You can define functions inside a function.

If you are very pedantic, you can say that Functional Programming is the implementation of *Lambda Calculus*, which is "a formal system in mathematical logic for expressing computation based on function abstraction and application using variable binding and substitution." This explains why FP style has a heavy "mathy" predisposition on approaching problems.

But then again, do I *look* that old? Carpe diem! Let's go through some actual, fun examples of Functional Programming, so we can ditch the textbook answers. Books are for squares! Also, in this notebook we will be tempered in our approach to programming: when to use functional programming and when to mix functional programming with OOP.

## Something or Nothing (None)
In Python, functions (and all callables like methods and classes) always return something. Usually, you return some calculated result. Other times, your function is really running a check on the inputs and doesn't return anything (or does it?). If your function does not explicitly return something, then it returns `None`.

In [1]:
def double(x): # returns something useful
    return x * 2

In [2]:
def is_even(x): # returns None
    if x % 2 == 0:
        print("Life is good")
print(is_even(10)) # None is printed

Life is good
None


In [3]:
# all logically equivalent
def do_nothing1():
    pass

def do_nothing2():
    return

def do_nothing3():
    return None

print(do_nothing1()) # None is printed
print(do_nothing2()) # None is printed
print(do_nothing3()) # None is printed

None
None
None


In [4]:
# identical bytecode for each of the functions
import dis

print(dis.dis(do_nothing1))
print(dis.dis(do_nothing2))
print(dis.dis(do_nothing3))

  3           0 LOAD_CONST               0 (None)
              2 RETURN_VALUE
None
  6           0 LOAD_CONST               0 (None)
              2 RETURN_VALUE
None
  9           0 LOAD_CONST               0 (None)
              2 RETURN_VALUE
None


In [5]:
print(print("HI")) # print() itself is a function, so it returns something--None

HI
None


## `map`/`filter`/`reduce`
List comprehensions are generally the "Pythonic" way to do things. Let's try to do the same using FP syntax.  A list comprehension can do both `map` and `filter` and the reverse is true: all list comprehensions are a combination of `map` and `filter`. Instead of updating/modifying a list, just return a new one.

FP does *not* use (explicit) loops; instead FP uses `map`/`filter`/`reduce` (and recursion) to achieve the same results as loops. Also remember the added bonus is that `map` and `filter` are lazy (do not immediately execute so saves on runtime) and uses low memory footprint. `reduce` is eagerly/immediately executed but also uses low memory footprint.

In [1]:
# double element: OOP where mutating/updating a list (using a method)
accumulator = [] # the list is being mutated
for i in range(10):
    accumulator.append(i * 2)
accumulator

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [2]:
# double element: FP equivalent with no mutations
list(map(lambda x: x * 2, range(10)))

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [3]:
# remove evens: OOP where mutating/updating a list
accumulator = []
for i in range(10):
    if i % 2:
        accumulator.append(i)
accumulator

[1, 3, 5, 7, 9]

In [4]:
# remove evens: FP equivalent with no mutations
list(filter(lambda x: x % 2, range(10)))

[1, 3, 5, 7, 9]

A list comprehension cannot be used to emulate `reduce`. You'll need a for-loop with an accumulator to get the equivalent of FP's `reduce`. However, the accumulator is stateful (by having to explicitly store a variable) whereas `reduce` does not have any global variable that kept between function calls.

Note: Guido van Rossum does not like `reduce` and prefers an explicit for-loop. That is why `reduce` is no longer a builtin function in Python 3 and thus has to be imported from `functools`.

In [5]:
# imperative approach
accumulator = 0
for i in range(10):
    accumulator += i
accumulator

45

In [6]:
# FP: not need for statefulness in the form of an intermediate/temporary accumulator variable
from functools import reduce

reduce(lambda x, y: x + y, range(10))

45

In [7]:
# reduce can take a 3rd argument where it is an initializer. Useful when the iterable is empty
reduce(lambda x, y: x + y, [], 42)

42

When you are given a hammer, everything looks like a nail. Use Functional Programming when it benefits you. Also, learn when to not force FP into a problem that it is less appropriate. There are some problems that lends itself to OOP and there are some that lend themselves to FP. Let me give you an example where OOP is far superior.  
Always keep in mind the Golden Rule of programming? ~~Do onto others whatever you would like them to do to you.~~ **Use the best tool for the problem.**

In [8]:
from string import ascii_letters
from random import choices

print(ascii_letters)
lots_of_letters = choices(ascii_letters, k=100000)

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ


In [9]:
%%time
result_OOP = "".join(lots_of_letters) # OOP is much faster since runtime is O(n)

Wall time: 0 ns


In [10]:
%%time
result_FP = reduce(lambda x, y: x + y, lots_of_letters) # FP is much slower since runtime is O(n ** 2)

Wall time: 252 ms


In [11]:
result_OOP == result_FP

True

### Reach for the Stars: `starmap`
1 weakness of a regular `map` function is it usually takes 1 argument: the element itself. One way to add an additional argument is to take the other arguments and hard code them into the `lambda` function inside the `map`. However, what if the second arguments keep changing? The other (and preferred) way is to use `starmap()`. You can think of `starmap` as `map(*args)`.

In [12]:
import itertools

list(
    itertools.starmap(
        lambda x, y: x * y, 
        [(1, 2), (3, 4), (5, 6)], # second argument is list of "pre-zipped" arguments
    )
)

[2, 12, 30]

In [13]:
# here's a rough implementation of starmap
def starmap(function, iterable):
    for args in iterable:
        yield function(*args)

list(
    starmap(
        function=lambda x, y: x * y, 
        iterable=[(1, 2), (3, 4), (5, 6)],
    )
)

[2, 12, 30]

## Unstoppable Force Meets an <i>Immutable</i> Object
Immutability means that an object cannot be changed or updated. Many useful objects are mutable: list, dict, sets. Other useful objects are immutable: string and bytes, numbers (float, int), bool, None, tuples. For example, you cannot make a 1 equal a 2 no matter how hard you try. How do you test if an object is updated or an entirely a new one? Check for identity using `id()` (which gets memory address) or `is`. Since immutable types do not have updates, they do not have the methods: `pop()`, `append()`, `add()`, `update()`.

In [1]:
l = []
print(id(l), l)
l.append(1)
print(id(l), l) # same object before and after
l += [2]
print(id(l), l) # same object before and after

1658661072328 []
1658661072328 [1]
1658661072328 [1, 2]


Let's show that an immutable object's identity changes every time you rebind the variable to new data.

In [2]:
x = 9000
print(id(x), x)
x += 1
print(id(x), x) # totally different object

1658661570256 9000
1658661570352 9001


`bool` only has 2 instances: `True` and `False`. Thus, all `False`s are the SAME `False` and not only are they equivalent to each other, they are 1 and the same object--they are `id`entical to each other--in other words, there is only 1 `False`.

`None` is a singleton--there is only 1 unique `None` instance of the `NoneType` class. All `None`s are identical to each other. For checking identity, use the `is` operator. For example, you often see the code `if x is None:`--the reason is `is` for checking identity while `==` is for checking value equivalence.  
Note: the reason why we don't use `if x is True:` (even though this is logically correct) is that the `if` operator automatically casts the object to a `bool`. Hence, `if x:` is sufficient.

In [3]:
x = True
y = not False
z = bool(10)
print(id(x), x)
print(id(y), y)
print(id(z), z)
print(x == y == z)
print(x is y is z)

1794306208 True
1794306208 True
1794306208 True
True
True


In [4]:
for i in range(5):
    print(id(None))
print(None is None is None is None is None is None is None is None is None is None is None) # they are exactly the same object
print(
    None
    is (lambda: print())() # print returns None
    is type(None)() # get the class and then create an instance of the class
)

1794352272
1794352272
1794352272
1794352272
1794352272
True

True


`is` and `==` are not entirely equivalent.

In [5]:
print([] is [])
print([] == [])

False
True


What I mean by identity check is this:

In [6]:
# immutable object
x = None
y = None
print(x is y)
print(id(x) == id(y)) # logically equivalent to the previous line

True
True


In [7]:
# mutable object
x = []
y = []
print(x is y)
print(id(x) == id(y)) # logically equivalent to the previous line

False
False


In [8]:
# mutable object
x = [[1]] * 10
print(x)
x[0][0] = None
print(x) # everything changed! not expected

[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1]]
[[None], [None], [None], [None], [None], [None], [None], [None], [None], [None]]


In [9]:
# mutable object
x = [[1] for i in range(10)]
print(x)
x[0][0] = None
print(x) # only 1 element changed

[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1]]
[[None], [1], [1], [1], [1], [1], [1], [1], [1], [1]]


In [10]:
# real life problem I struggled with over 2 hours
import numpy as np

arr1 = np.array([[1, 2, 3], [4, 5, 6]])
arr2 = arr1 # I (mistakenly) thought I made a copy
arr1[0][0] = 9
print(arr1)
print(arr2) # also changed

[[9 2 3]
 [4 5 6]]
[[9 2 3]
 [4 5 6]]


In [11]:
# made a "copy" this time: problem is it was only a shallow copy
import copy

l1 = [[1, 2, 3], [4, 5, 6]]
l2 = copy.copy(l1) # l1.copy()
print(l1 is l2, id(l1), id(l2)) # they are 2 different objects but not exactly decoupled

l1.append(None)
print(l1)
print(l2) # unchanged, whew!

l1[0][0] = 9
print(l1)
print(l2) # changed, why?

False 1658665249736 1658665249800
[[1, 2, 3], [4, 5, 6], None]
[[1, 2, 3], [4, 5, 6]]
[[9, 2, 3], [4, 5, 6], None]
[[9, 2, 3], [4, 5, 6]]


### Small Detour on Copying
Python has 2 ways to copy objects: shallow and deep.

**Shallow copy**: copies all the pointers/memory address of the underlying objects in a data structure (ie list). However, if you have a list within a list, only the inner list's memory address is copied--not the elements themselves. You get the problem shown above.

**Deep copy**: Recursively copy all pointers to the underlying objects, including nested data structures and their elements. Once you made a deep copy, the original object is completely decoupled from the copy--you can do anything to the original object and there will be changes to the copy. The problem is deep copying is a relatively slow operation.

In [12]:
l1 = [[1,2,3], [4,5,6]]
l2 = copy.deepcopy(l1)
print(l1 is l2, id(l1), id(l2)) # they are 2 different objects but completed decoupled

l1.append(None)
print(l1)
print(l2) # unchanged, whew!

l1[0][0] = 9
print(l1)
print(l2) # unchanged, yes!

False 1658661072392 1658661072840
[[1, 2, 3], [4, 5, 6], None]
[[1, 2, 3], [4, 5, 6]]
[[9, 2, 3], [4, 5, 6], None]
[[1, 2, 3], [4, 5, 6]]


### In conclusion:
There are some confusion rules about assignment that makes the mutability of an object a liability. Updating an object is more confusing than creating a brand new object. Functional programming avoids/solves this problem by creating new objects instead of mutating an existing object.

## Is Your Function Pure of Heart?
What is a `pure` function vs `idempotent` function vs functions that are neither?

**Pure Function**: A function is pure if it always returns the same outputs given the same inputs. Also, a pure function does not mutate the state of things outside of its scope--ie no side effects. Pure functions are easier to reason about and test because they only do 1 thing and are totally decoupled between function calls. Nothing about the last function call is remembered--thus is stateless.

**Idempotent Function**: A function that always returns the same outputs given the same inputs. An idempotent function may change variables outside of its scope. A pure function is subset of idempotent function, so all pure functions are idempotent functions but not vice versa.

In [1]:
def double_pure(x): # a pure function; input always has a deterministic output. Nothing else changes
    return x * 2

In [2]:
def double_idempotent(x): # has side effect, by doing something outside of its scope: assigning "Hello" to y in global scope
    global y
    y = "Hello"
    return x * 2

print(double_idempotent(1), y)

2 Hello


In [3]:
def not_pure_nor_idempotent(x): # has side effect of mutating the object
    x.append(None)

simple_list = []
not_pure_nor_idempotent(simple_list)
print(simple_list)
not_pure_nor_idempotent(simple_list)
print(simple_list)
not_pure_nor_idempotent(simple_list)
print(simple_list)

[None]
[None, None]
[None, None, None]


## ~~Functions that are High~~ Higher-Order Functions
The previous examples are definitional and more academic. Let's go through 1 last definition and then we'll see the fun, applied use cases.

**High-order function**: A function that either takes in a function as an argument and/or returns a function.

In [1]:
def run_op(func, x): # takes in a function
    return func(x)

run_op(sum, range(10))

45

In [2]:
def add_two(): # returns a function
    return lambda x: x + 2

print(add_two()) # output is a function, not a value
print(add_two()(42)) # call the returned function

<function add_two.<locals>.<lambda> at 0x0000024BA6FC4BF8>
44


In [3]:
import time

def time_func(func): # takes in a function and returns a function
    def inner(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print("Runtime: {}".format(end_time - start_time))
        return result
    return inner

print(time_func(sum)) # output is a function, not a value
print(time_func(sum)(range(10))) # call the returned function

<function time_func.<locals>.inner at 0x0000024BA6FC4A60>
Runtime: 0.0
45


## Music to My Ears: Decorate and Compose
Decorators are introduced in `1_Pragmatic_Python.ipynb`. Nonetheless, here's a refresher on decorators: you have a function that you want to change some behavior before or after the function call. Basically you want to compose a function that alter the behavior of the original function.

Now you may be wondering: why would you want a decorator if you can just change the original function (if all you are going to do is change something before and/or after the original function call). The reason could be:
* You can't change the original function because it is too complicated to understand.
* You can't change the original function because you don't have access to change it. For example, many numpy functions are written in C, so you literally could not change the source code.
* You can't change the original function because it is used everywhere and you don't know which ones you want to change.
* You just wanna be ~~cool~~ a pro!

Note: be careful. A decorator is not exactly function composition. A decorator is not exactly f(g(x)) but something close. If you want to see a function composition example, then look towards the bottom of this notebook or just search for "Function Composition".


### My Variables and I Need Some Closure
Quick thing. A decorator is closure. Closure is a function that has access to variable(s) defined <i>outside</i> of the function. Closure is called closure since it has access to variables that are not locally defined, which are "closed over" by the closure function. This academic definition is confusing even though the idea is actually quite simple. Here's a simple example.

In [1]:
def outer():
    x = "they captured me"
    y = "they are closing in"
    def inner():
        print(x, y) # x and y are not defined or passed in as arguments to inner()
    return inner

inner = outer()
print(inner)
inner()

<function outer.<locals>.inner at 0x000001D7C81719D8>
they captured me they are closing in


Closure binds late. *Late binding* means that the value of a variable is only looked up at runtime. Basically, the value of a closed over variable is only determined when the function is called.  
Note: Late binding follows the LEGB rule. See `6_Pedantic_Python_Tricks.ipynb` to learn more about the LEGB rule.

Again the technical definition sounds confusing. An easy example will be illustrate the point better.

In [2]:
def outer():
    variable = "hi" # notice this value is never printed
    
    def inner():
        print(variable) # this variable is only looked up when this function is run

    variable = "thanks"
    inner() # look up variable at this point in time
    
    variable = "bye"
    inner() # look up variable at this point in time

outer()

thanks
bye


Now back to decorators. A decorator is closure since you pass the function (you want to decorate) to the outer function, but that same function is accessable in the inner function without being passed as an argument into the inner function. Decorators allow you to do something *before* and/or *after* your original function is called. You can also change the arguments passed into your original function, so you may consider that effect as <i>"during"</i> your original function call. In short, a decorate alters the original function call *before* and/or *after* and/or *during*.

In [3]:
def make_my_function_polite(func):
    def inner(arg):
        print("hello!") # before
        result = func(arg.upper()) # during; also notice I'm storing results if I want to do something after my function call
        print("bye!") # after
        return result
    return inner # notice I am returning a function back

def double(x):
    return x * 2

In [4]:
make_my_function_polite(double) # notice that a function is returned

<function __main__.make_my_function_polite.<locals>.inner>

In [5]:
make_my_function_polite(double)("thanks") # apply decorator 1 time

hello!
bye!


'THANKSTHANKS'

The Pythonic syntax is to use the `@` sign instead of manual function call (on original function and assigning back to original function name). The following 2 cells are logically equivalent. The first cell is the preferred syntax.

In [6]:
def make_my_function_polite(func):
    def inner(arg):
        print("hello!") # before
        result = func(arg.upper()) # during; also notice I'm storing results if I want to do something after my function call
        print("bye!") # after
        return result
    return inner # notice I am returning a function back

@make_my_function_polite
def double(x):
    return x * 2

double

<function __main__.make_my_function_polite.<locals>.inner>

In [7]:
def make_my_function_polite(func):
    def inner(arg):
        print("hello!") # before
        result = func(arg.upper()) # during; also notice I'm storing results if I want to do something after my function call
        print("bye!") # after
        return result
    return inner # notice I am returning a function back

def double(x):
    return x * 2

double = make_my_function_polite(double) # decorator function call on original function and then assign to original function name
double

<function __main__.make_my_function_polite.<locals>.inner>

In fact, you can even stack/chain decorators. The decorator closest to the function the decorator is applied first. Now that I think about it, a decorator is in some sense like a generator but for functions--decorators change the function call at execution time but are effectively lazy in that nothing appears to change right now.

In [8]:
def greetings__I_am_close(func):
    def inner(arg):
        print("Hiiiiii! -Kirby")
        return func(arg)
    return inner
    
def farewell__I_am_far(func):
    def inner(arg):
        results = func(arg)
        print("I'll be back! -The Terminator")
        return results
    return inner

@farewell__I_am_far # this wraps second
@greetings__I_am_close # this wraps the original function first
def double(x):
    return x * 2

double(42)

Hiiiiii! -Kirby
I'll be back! -The Terminator


84

Now you have another question: what if you don't know the number of argments or argument names in ahead of time for the decorated function? That's where you use `*args` and `**kwargs`! This setup covers all cases.

If you need a refresher on `*args` and `**kwargs`, you can take a look at `1_Pragmatic_Python.ipynb`. Basically, `*args` is for a variable (and indeterminate) number of positional arguments and `**kwargs` is for a variable (and indeterminate) number of keyword arguments. Depending on where they are used, they can either separate out the arguments or gather them together. If they are used at function call time, then `*args` and `**kwargs` will separate your list/tuple or dictionary of arguments correctly into the function's arguments. If you are using them at function definition time, then they will gather your arguments into a tuple or dictionary.  
Brief note: `*args` is pronounced phonetically: star args. `**kwargs` is also phonetic: star star kwa-args.

In [9]:
def identity_decorator(func): # does nothing special
    def inner(*args, **kwargs):
        return func(*args, **kwargs)
    return inner

@identity_decorator
def silly_function(a, b, c=3):
    print(a, b, c)
    
silly_function(1, 2)
silly_function(a=5, b=4)
silly_function(*(6, 7, 8))
silly_function(**{"c": 10, "a": 11, "b": 12})

1 2 3
5 4 3
6 7 8
11 12 10


Fun fact: `*args` is called splatting in other languages. It is useful for transposing a matrix.

In [10]:
list(zip(*[
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]])
)

[(1, 4, 7), (2, 5, 8), (3, 6, 9)]

You can use `*args` and `**kwargs` multiple times in the same function call.

In [11]:
list(
    zip(*[
        [1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]
        ],
        [10, 11, 12],
       ),
)

[(1, 4, 7, 10), (2, 5, 8, 11), (3, 6, 9, 12)]

In [12]:
list(
    zip(
        *[
            [1, 2, 3],
            [4, 5, 6],
            [7, 8, 9]
        ],
        *[
            [10, 11, 12],
            [13, 14, 15],
            [16, 17, 18],
        ]
    ),
)

[(1, 4, 7, 10, 13, 16), (2, 5, 8, 11, 14, 17), (3, 6, 9, 12, 15, 18)]

If you use `**kwargs`, the keys must be valid argument names, ie the keys must be strings that start with an alphabetical character and has no spaces.

In [13]:
dict(**{"1": 1, "2": 2, "3": 3}, **{"4": 4, "5": 5, "6": 6})

{'1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6}

Now you may ask. What about the docstring of the decorated function? Won't the outputted function contain the decorator's docstring instead of the original decorated function's docstring? You are correct. That's why we use `functools.wraps()` decorator to extract the correct docstring from the decorated function and apply it to the outputted function.

In [14]:
def identity_decorator(func):
    """This docstring is totally lost after decorating a function"""
    def inner(*args, **kwargs):
        """Decorator that does nothing special"""
        return func(*args, **kwargs)
    return inner

@identity_decorator
def double(arg):
    "Double the value" # this docstring is lost
    return arg * 2


help(double)

Help on function inner in module __main__:

inner(*args, **kwargs)
    Decorator that does nothing special



In [15]:
from functools import wraps

def identity_decorator(func):
    """This docstring is totally lost after decorating a function"""
    @wraps(func) # I decorate the inner function, so that I extract the original function's docstring and apply it to the returned inner() function
    def inner(*args, **kwargs):
        """Decorator that does nothing special""" # now this docstring is suppressed
        return func(*args, **kwargs)
    return inner

@identity_decorator
def double(arg):
    "Double the value" # now this docstring is kept
    return arg * 2


help(double)

Help on function double in module __main__:

double(arg)
    Double the value



Now you may have a question. Does decorators work on things outside of functions? YES! Decorators also works on classes or, in general, anything that is callable.

In [16]:
import pandas as pd

def polite_class(cls):
    def inner(*args, **kwargs):
        print("Greetings, I come in peace --Never said by the Borg")
        result = cls(*args, **kwargs) # technically you don't need the save it; you can just
        return result # return it if you don't do anything special after calling the class
    return inner

pd.DataFrame = polite_class(pd.DataFrame) # function decorating a class
display(pd.DataFrame({"a": range(5)}))

@polite_class
class A:
    pass

print(A())

Greetings, I come in peace --Never said by the Borg


,a
0,0
1,1
2,2
3,3
4,4


Greetings, I come in peace --Never said by the Borg


Now you may another question: can decorators take any arguments other than the target function? YES! Remember, a decorator itself is just a regular function. The only difference is that instead of taking input arguments of raw data and returning processed/outputted data, a decorator takes in function and return a "processed/enhanced/augmented" function. That is why it is called a decorator--it decorates the original function.

The decorator example below is a simple to way to make Python enforce type on the arguments. You can think of it as making Python (pseudo) statically typed. It is not real static typing (because Python does not have a compilation step) and because the invalid argument types are only caught at runtime (when you call the function).

In [17]:
def check_type(type_): # though "type" is not a reserved word, it is better to use a non-conflicting variable name
    def outer(func): # Boo! Surprise! The decorator is really here
        def inner(arg): # both`type_` and `func` are closed over by inner()
            if isinstance(arg, type_):
                return func(arg)
            else:
                raise Exception("arg {} is not of type {}".format(arg, type_))
        return inner
    return outer
                
@check_type(int)
def is_even(x):
    return x % 2 == 0

print(is_even(10)) # prints True
print(is_even(10.0)) # raises Exception

True


Exception: arg 10.0 is not of type <class 'int'>

## Recursion: Turtles All the Way Down
Functional programming does not use explicit `for` or `while` loops--FP emulates them by using `map` and `filter`. However, some computations exceed the ability of only `map` and `filter`. And that's when recursion comes in. Recursive functions *call* themselves!

Sometimes, you don't know how to solve a big problem, but you know how to solve a little problem. If you solve the little problem, then the bigger problem naturally gets solved. Recursion is solving through induction rather than a head-on, direct approach. (Look at all the math majors smiling 😍)

In a recursive function, there are 2 cases: base case and recursive case.
* **Base case**: when you have arrived at a fundamental stopping point because you solved the innermost, smallest problem such that the function no longer needs to call itself (non-recursive). You can have multiple base cases. Generally, you put the base case at the top of the function's body.
* **Recursive case**: when you still have smaller problems to solve, so you call the function itself. Every call of the recursive case should get you closer to the base case. You can have multiple recursive cases. Generally, you put the recursive case at the bottom of the function's body.

NOTE: Recursion and iteration(ie loops) are 2 different approaches to repetition, and both are Turing Complete. Hence, both are equally expressive: any recursion can be written as iteration and vice versa. Hence, FP and Procedural Python are equally expressive--it is a matter of your preference.

Rather than any more academic explanations, let's go through the "hello world" example of recursion: factorial.

In [1]:
def factorial(n):
    if n == 1: # base case
        return 1
    else: # recursive case
        return n * factorial(n-1)

# 1 liner equivalent if you're fancy
# def factorial(n):
#     return n * factorial(n-1) if n != 0 else 1

factorial(5)

120

However, there's a limitation in how deep you can recurse. The reason is that suppose you wrote your recursion wrong and it never hits the base case, then the function keeps call itself infinitely--I guess you can call FP's infinite recursion as analogous to procedural programming's infinite `while` loop (where it is always true and can never execute the loop). To guard against that, Python has a maximum call stack depth, which limits how many nested function calls you can do. The call stack is made out of a stack frames, one for each nested function call.

![SegmentLocal](images/recursion_call_stack.gif "segment")
Notice that the 1st function call is at the bottom and then the recursive calls builds the call stack **up**--you are stacking things up and the things that are being stacked are each function's call. Once your function hits the base case, the result is discovered, which passes result to the function call (the one before/below in the call stack) that created this stack frame. Thus, the call stack "pops" off the stack frame that is resolved, so the stack is decreasing. Once the stack frame hits the bottom, your 1st function call returns you the actual result.  
Note: animation attributed to https://realpython.com/python-thinking-recursively/

In [2]:
import inspect

def factorial_with_stack_depth(n):
    print("entering stack depth: {}".format(len(inspect.stack()) - current_depth))
    if n == 1:
        return 1
    else:
        result = n * factorial_with_stack_depth(n-1)
        print("exiting stack depth: {}".format(len(inspect.stack()) - current_depth))
        return result

current_depth = len(inspect.stack())
factorial_with_stack_depth(5) # stack goes up and then down

entering stack depth: 1
entering stack depth: 2
entering stack depth: 3
entering stack depth: 4
entering stack depth: 5
exiting stack depth: 4
exiting stack depth: 3
exiting stack depth: 2
exiting stack depth: 1


120

In [3]:
import sys

sys.getrecursionlimit()

3000

In [4]:
%%time
factorial(2000) # super fast

Wall time: 5 ms


3316275092450633241175393380576324038281117208105780394571935437060380779056008224002732308597325922554023529412258341092580848174152937961313866335263436889056340585561639406051172525718706478563935440454052439574670376741087229704346841583437524315808775336451274879954368592474080324089465615072332506527976557571796715367186893590561128158716017172326571561100042140124204338425737127001758835477968999212835289966658534055798549036573663501333865504011720121526354880382681521522469209952060315644185654806759464970515522882052348999957264508140655366789695321014676226713320268315522051944944616182392752040265297226315025747520482960647509273941658562835317795744828763145964503739913273341772636088524900935066216101444597094127078213137325638315723020199499149583164709427744738703279855496742986088393763268241524788343874695958292577405745398375015858154681362942179499723998135994810165565638760342273129122503847098729096266224619710766059315502018951355831653578714922909167790497022470

Despite the following code has a terminating point and is logically correct, Python fails and triggers `RecursionError` exception.

In [5]:
factorial(4000) # maximum depth exceeded

RecursionError: maximum recursion depth exceeded in comparison

The solution to that is to either increase the maximum depth size or do something smarter. Increasing maximum depth size is very easy, so let's think of how to approach the problem in a smarter way.

### Tail Call Optimization
Tail call optimization (also called tail call elimination) is a way that FP languages "beat" the call stack by not creating another stack frame when the last function call is solely itself--the next call uses the existing stack frame. Often, tail call optimization uses an *accumulator* variable.

BTW, Python does **not** have tail call optimization, so why am I telling you all this technobabble when Python doesn't even support it? Because I want to help you see the problem in another way such that the solution becomes tractable. Here's what `factorial()` would look like IF written in a tail call style.

In [6]:
def factorial_tco(n, accumulator=1):
    if n == 1: # base case
        return accumulator
    else: # recursive case
        return factorial_tco(n-1, accumulator=accumulator*n) # the return only has a function call of itself
        # The previous implementation `return n * factorial(n-1)` is not a tail call since it contains `n * ...`

factorial(5)

120

However, tail call optimization syntax is really logically equivalent to a `while` loop. So let's write the Procedural Python equivalent.

In [7]:
def factorial_loop(n):
    accumulator = 1
    while n:
        accumulator *= n
        n -= 1
    return accumulator

factorial_loop(5)

120

So basically... you taught Tail Call Optimization just to tell me to write a recursive function as a procedural function (with a loop)? What a cop out! I know 🙂. If you need to recurse very deeply, you will probably want to use a loop (or another optimization). If your recursion is only hundreds deep, then just use recursion if you feel like it.

Let's take a look at another beginner recursive function: fibonacci. The next number in the sequence is the sum of the previous 2 numbers.

In [8]:
def fibonacci_plain(n):
    if n == 1: # base case 1
        return 1
    elif n == 2: # base case 2: you can merge with base case 1 if you want
        return 1
    else: # recursive case
        return fibonacci_plain(n-1) + fibonacci_plain(n-2)

# functionally equivalent 1 liner
# def fibonacci(n):
#     return fibonacci(n-1) + fibonacci(n-2) if n not in (1, 2) else 1

fibonacci_plain(8)

21

Here's the tail call optimization version. Notice the accumulator variable. I find this version harder to understand.

In [9]:
def fibonacci_tco(n, previous=1, current=1):
    if n == 1 or n == 2:
        return current
    else:
        return fibonacci_tco(n - 1, current, previous + current)

fibonacci_tco(8)

21

Here's the procedural iterative version. Notice the accumulator variable. I find this version easier to understand.

In [10]:
def fibonacci_loop(n):
    if n == 1 or n == 2:
        return 1
    else:
        previous, current = 1, 1
        for i in range(n - 2):
            previous, current = current, previous + current # `current` is the accumulator variable
    return current

fibonacci_loop(8)

21

If you are really fancy, you can go full circle and transform the iterative style back to FP style with `reduce()`. I would have never thought of this version naturally.

In [11]:
from functools import reduce

n = 8
reduce(lambda previous_current, _: (previous_current[1], sum(previous_current)), range(n-2), (1, 1))[1]

21

Everybody does fibonacci. Let's do the lesser known variant: tribonacci. The next number in the sequence is the sum of the previous 3 numbers.

In [12]:
def tribonacci_plain(n):
    if n == 1: # base case 1
        return 1
    elif n == 2: # base case 2
        return 1
    elif n == 3: # base case 3
        return 2
    else: # recursive case
        return tribonacci_plain(n-1) + tribonacci_plain(n-2) + tribonacci_plain(n-3)

tribonacci_plain(8)

44

The iterative way is easier to understand.

In [13]:
def tribonacci_loop(n):
    if n == 1 or n == 2:
        return 1
    elif n == 3: # base case 3
        return 2
    else:
        previous1, previous2, current = 1, 1, 2
        for i in range(n - 3):
            previous1, previous2, current = previous2, current, previous1 + previous2 + current
        return current

tribonacci_loop(8)

44

Now let's try a more fun example: Collatz conjecture, which states that if you follow these 2 rules, you should always end up with the number 1.
* If the number is even, divide it by two.
* If the number is odd, triple it and add one.

So let's use recursion to figure out how many steps it takes to reach 1.

In [14]:
def collatz(n, steps=None): # steps is the accumulator
    steps = steps if steps else []
    steps += [n]
    if n == 1:
        return steps
    else:
        if n % 2: # odd: recursive case 1
            return collatz(n * 3 + 1, steps=steps)
        else: # even: recursive case 2
            return collatz(n // 2, steps=steps)

print(collatz(10))
print(collatz(19))

[10, 5, 16, 8, 4, 2, 1]
[19, 58, 29, 88, 44, 22, 11, 34, 17, 52, 26, 13, 40, 20, 10, 5, 16, 8, 4, 2, 1]


Just a reminder: FP's recursion is just another way to view procedural loops--the 2 styles form a duality, 2 sides of the same coin. It's up to you to decide which style you prefer.

## Academic Section: You Can Skip To the Next Section If You Like
**Currying**, **Partial Functions**, and **Function Composition** are part of FP as they are higher-order functions, but I don't use them in real life. They are more for theoretical interest and perhaps more useful in other languages.

### Currying: No, Not the Japanese Dish
If a function has multiple arguments, currying is actually separating the arguments into separate function calls. It isn't exactly function decomposition, but it is decomposing a function by its arguments. You generally know the number of arguments of the target function when you are using currying. Currying is as exactly as expressive as the original function--ie no loss of expressiveness. A curried function is called by applying the arguments 1 by 1.

In [1]:
def decomposer(func):
    def inner1(a):
        def inner2(b):
            def inner3(c):
                return func(a, b, c)
            return inner3
        return inner2
    return inner1

@decomposer
def multiplier(x, y, z):
    return x * y * z

print(multiplier(1)) # function
print(multiplier(1)(2)) # function
print(multiplier(1)(2)(3)) # actual result

<function decomposer.<locals>.inner1.<locals>.inner2 at 0x000001AD8D270708>
<function decomposer.<locals>.inner1.<locals>.inner2.<locals>.inner3 at 0x000001AD8D270A68>
6


### Partial Function Application: Impatially Filling in the Blanks
A partial function application freezes/hardcodes/fills in 1 or more arguments of the original function. The outputted function now requires fewer arguments to actually run. The goal to build more nuanced function(s) from a base/more general function. The outputted function is less expressive than the original function.

In [2]:
from functools import partial

def multiplier(x, y):
    print("x is {}; y = {}".format(x, y))
    return x * y

multiply_by_2 = partial(multiplier, 2) # setting position arguments, equivalent to *args
multiply_by_3 = partial(multiplier, 3)

print(multiply_by_2(5))
print(multiply_by_3(5))

x is 2; y = 5
10
x is 3; y = 5
15


In [3]:
# you can also set keyword arguments, equivalent to **kwargs
multiply_by_2 = partial(multiplier, y=2)
multiply_by_3 = partial(multiplier, y=3)

print(multiply_by_2(5))
print(multiply_by_3(5))

x is 5; y = 2
10
x is 5; y = 3
15


In [4]:
# here's a simple implementation of partial of filling 1 or multiple (positional or named) arguments
def partial(func, *_args, **_kwargs):
    if (not _args) and (not _kwargs):
        raise Exception("No arguments filled")
    else:
        def inner(*args, **kwargs):
            kwargs.update(_kwargs)
            return func(*_args, *args, **kwargs)
    return inner

def multiplier(x, y, z):
    return x * y * z

In [5]:
multiply_by_2 = partial(multiplier, 2)
multiply_by_2(3, 4)

24

In [6]:
multiply_by_6 = partial(multiplier, 2, 3)
multiply_by_6(4)

24

In [7]:
multiply_by_12 = partial(multiplier, y=3, z=4)
multiply_by_12(2)

24

In [8]:
multiply_by_8 = partial(multiplier, 2, z=4)
multiply_by_8(3)

24

### Function Composition: Build me a Skyscraper out of Lego Bricks
The mathy part of FP. If you remember high school, you might have seen this funky notation: (f ∘ g)(x), which just means f(g(x)). Notice that the order of the application of the functions; the closer function is applied first. 
Suppose $f(x) = 3x + 2$ and $g(x) = x + 5$.

In [9]:
# Here's the procedural style
def f(x):
    return x + 5

def g(x):
    return 3*x + 2

input_value = 42
intermediate_value = g(input_value)
final_value = f(intermediate_value)
final_value

133

In FP, you can compose a higher-order function. Here are 2 options:

In [10]:
# Here's the decorator style
def f(func):
    def inner(input_value):
        return func(input_value) + 5
    return inner

def g(x): # same as above
    return 3*x + 2

h = f(g)
input_value = 42
final_value = h(input_value)
final_value

133

In [11]:
# Here's the compose function
def compose(f, g):
    return lambda x: f(g(x))

def f(x):
    return x + 5

def g(x):
    return 3*x + 2

h = compose(f, g)
input_value = 42
final_value = h(input_value)
final_value

133

Suppose you want to chain together more than 2 functions, the compose style still works.

In [12]:
# Here's the compose function on multiple functions
def compose(f, g):
    return lambda x: f(g(x))

def f(x):
    return x + 5

def g(x):
    return 3*x + 2

def h(x):
    return x * x

i = compose(f, compose(g, h))
input_value = 42
final_value = i(input_value)
final_value

5299

And if you are a fancy pants 🧐, here's a magic trick ✨. It's just for fun. Don't think about it too hard--unless you want to. 😃

In [13]:
# Here's the generalized compose function (for multiple functions)
from functools import reduce

def compose(list_of_functions):
    def inner(x):
        return reduce(lambda value, func: func(value), reversed(list_of_functions), x)
        # Notice how I am using reduce(). reduce() is usually used to apply 1 function
        # to multiple pieces of data, but here I am applying 1 piece of data to multiple
        # functions iteratively.
    return inner

def f(x):
    return x + 5

def g(x):
    return 3*x + 2

def h(x):
    return x * x

i = compose([f, g, h])
input_value = 42
final_value = i(input_value)
final_value

5299

## Python's Functions, Not Necessarily Functional Programming
Let me explain some cool things about **Python's** functions that might not be strictly applicable to FP.

### Functions are First Class
When Python says that functions are first-class citizens, it means you can do (virtually) anything with functions. Functions are at the end of a day just another object. Functions can be used as an input argument and as a function output as seen in higher-order functions, decorators, currying, partial function application, etc.

Python's function can be assigned, hashable, have their own attributes, and lots of other stuff.

In [1]:
# Functions can also be assigned
def func1():
    return 42

func2 = func1
print(func1 is func2)
print(func2())

True
42


In [2]:
# Functions are hashable and thus can be the key (and value) of a dict or element of a set.
# Basically functions can be stored in a data structure
print(hash(func1))
print({func1: func2}) # NOTE: FP does not use dict
print(set([func1, func2]))

-9223371875365723608
{<function func1 at 0x0000025997F62288>: <function func1 at 0x0000025997F62288>}
{<function func1 at 0x0000025997F62288>}


In [3]:
# Functions can even have their own attributes. See 3_Object_Oriented_Python.ipynb: "everything is an object"
# NOTE: FP probably does NOT allow functions to have attributes, as FP prefers closure instead.
def factorial(n):
    if not hasattr(factorial, "accumulator"):
        factorial.accumulator = 1
    if n == 1:
        return factorial.accumulator
    else:
        factorial.accumulator *= n
        return factorial(n - 1)

factorial(5)

120

### Cache Advance
We can make things faster with a cache and thus improve runtime efficiency. But first, what is a cache? Is it money? No, it's even better--it's an optimization trick!

A cache is just a place where you keep frequently-used data, so you can look it up quicky. For example, you can think of a sticky note with frequently asked questions and answers as a cache. In Python, a cache often uses a dictionary since it is very fast to see if a key exists in a dictionary--if so, get the value. If not, calculate the answer and update the dictionary. For example, a naive implementation of fibonacci has O(n^2) runtime complexity--it has quadratic runtime.

In [4]:
%%time
import time

def fibonacci_naive(num):
    print("current number is {}".format(num))
    time.sleep(0.1)
    if (num == 1) or (num == 2):
        return 1
    else:
        return fibonacci_naive(num - 1) + fibonacci_naive(num - 2)


fibonacci_naive(7)

current number is 7
current number is 6
current number is 5
current number is 4
current number is 3
current number is 2
current number is 1
current number is 2
current number is 3
current number is 2
current number is 1
current number is 4
current number is 3
current number is 2
current number is 1
current number is 2
current number is 5
current number is 4
current number is 3
current number is 2
current number is 1
current number is 2
current number is 3
current number is 2
current number is 1
Wall time: 2.51 s


13

A faster way is to use a cache to reuse already-computed results, so you don't have run unnecessary, duplicate computations. Since functions can have attributes, I am able to persist the results across function calls. Hence, I can reduce a naive recursion's O(n^2) runtime to a sophisticated recursion's O(n) runtime.

In [5]:
%%time
import time

def fibonacci_with_cache(num):
    try:
        if num in fibonacci_with_cache.cache:
            print("I looked up the answer for {}".format(num))
            return fibonacci_with_cache.cache[num]
    except AttributeError:
        fibonacci_with_cache.cache = {}        
    print("current number is {}".format(num))
    time.sleep(0.1)
    if (num == 1) or (num == 2):
        result = 1
    else:
        result = fibonacci_with_cache(num - 1) + fibonacci_with_cache(num - 2)
    fibonacci_with_cache.cache[num] = result
    return result


fibonacci_with_cache(7)

current number is 7
current number is 6
current number is 5
current number is 4
current number is 3
current number is 2
current number is 1
I looked up the answer for 2
I looked up the answer for 3
I looked up the answer for 4
I looked up the answer for 5
Wall time: 705 ms


13

In [6]:
%%time
# a simple implementation of a cache to be used as a decorator
def memoize(func):
    cache = {}
    def inner(num):
        if num not in cache:
            cache[num] = func(num)
        else:
            print("I looked up the answer for {}".format(num))
        return cache[num]
    return inner

@memoize
def fibonacci_memoized(num):
    print("current number is {}".format(num))
    time.sleep(0.1)
    if (num == 1) or (num == 2):
        return 1
    else:
        return fibonacci_memoized(num - 1) + fibonacci_memoized(num - 2)

fibonacci_memoized(7)

current number is 7
current number is 6
current number is 5
current number is 4
current number is 3
current number is 2
current number is 1
I looked up the answer for 2
I looked up the answer for 3
I looked up the answer for 4
I looked up the answer for 5
Wall time: 708 ms


13

But why build our own cache? Python has batteries included! With `lru_cache()` (which stands for Least-Recently-Used cache), we can calculate fibonacci with O(n) linear runtime; same as before but with a lot less code. `lru_cache()` has an argument for `maxsize`--how big do you want your cache to be? If the cache has more elements than its `maxsize`, then pop off the least-recently used key-value pair. That way, your cache never grows too large.

Decorators that have arguments (such as `lru_cache(maxsize)` are basically decorators within decorators. Nested decorators is how they are written: the outer decorator takes configuration(s) as arguments; the inner decorator takes in the target function as the argument.  
Just a side note: caches are useful in a technique called **memo**ization--not a typo, not **memor**ization. Memoization + recursion = dynamic programming.

In [7]:
%%time
from functools import lru_cache

@lru_cache(maxsize=128) # decorator takes an argument
def fibonacci_memoized_simplified(num):
    print("current number is {}".format(num))
    time.sleep(0.1)
    if (num == 1) or (num == 2):
        return 1
    else:
        return fibonacci_memoized_simplified(num - 1) + fibonacci_memoized_simplified(num - 2)

fibonacci_memoized_simplified(7)

current number is 7
current number is 6
current number is 5
current number is 4
current number is 3
current number is 2
current number is 1
Wall time: 703 ms


13

A cache is just something that remembers previous results, so it is debatable about whether memoization is considered true Functional Programming style since you are updating the cache. However, remember the 9th line of "The Zen of Python": practicality beats purity. Purity here does not refer to pure functions; rather it is the idea of not sticking to one programming paradigm dogmatically. Practicality refers to using the best tool for the problem. For example, sorting in a FP-style with pure functions would take a lot of memory and processing time. Sorting is more efficient in both memory and processing time by having a mutable data structure.

In [8]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


## Functional Fusion: Combining with OOP and Procedural
Sometimes, using pure FP style can be restricting. For example, UIs are hard to do in a purely FP style. Let's see how we can combine (or fuse) FP with other styles. The main use case for combining FP with other styles is high-performance Python (the next notebook!), which often uses method chaining and fluent interface. Dispatch is a nifty tool that you might see once in awhile. Infinite recursion depth is quite academic, so don't worry about it in real life.

### Daisy Chaining: Functional Programming + Method Chaining/Fluent Interface
Method chaining is **not** a FP style since method chaining belongs to OOP. However, let's take a look how method chaining simplifies the syntax for functional programming. `map(map(map(lambda ..., iterable), iterable), iterable)` looks ugly. Here's how we can separate the transformations onto sequential calls (on  separate lines) instead of nested calls in order to maximize readability. Like `|` in bash, treat iterables as a data pipeline where you are applying another transformation sequentially.

In [1]:
# using pandas map() method, which is element-wise apply() and takes only 1 argument (which is the element)
import pandas as pd
df = pd.DataFrame({"col": range(10)})

df["col"].map(lambda x: x * 2).map(lambda x: x % 3)

0    0
1    2
2    1
3    0
4    2
5    1
6    0
7    2
8    1
9    0
Name: col, dtype: int64

In [2]:
# pandas pipe() method can take multiple arguments, more flexible than map()
(
    df["col"]
    .pipe(lambda x: x * 2) # no extra arguments, equivalent to map/apply
    .pipe(lambda x, y: x * y, 3) # added a second argument, argument not named
    .pipe(lambda x, y, z: x * y * z, 4, z=5) # added (unnamed) second and (named) third argument
)

0       0
1     120
2     240
3     360
4     480
5     600
6     720
7     840
8     960
9    1080
Name: col, dtype: int64

In [3]:
# create new columns in dataframe without mutating existing dataframe.
# create new dataframe after each transform so like FP with pure function but use OOP syntax with method chaining
(
    df
    .assign(new_col1=lambda self: self["col"] * 2) # create new column
    .assign(new_col2=lambda self: self["col"] * 3) # create another new column based on the new column
    .assign(col=lambda self: self["col"] * 0) # overwrite original column
)

,col,new_col1,new_col2
0,0,0,0
1,0,2,3
2,0,4,6
3,0,6,9
4,0,8,12
5,0,10,15
6,0,12,18
7,0,14,21
8,0,16,24
9,0,18,27


Another library is `fluentpy` that gives you the expressiveness of FP but using the fluent interface. A fluent interface is a method that returns `self` after the method call. Basically you can chain a bunch of transformations together into 1 big expression instead of multiple statements (ie `temp1 = map(..., iterable); temp2 = filter(..., temp1); temp3 = reduce(..., temp3)`).  
Fluentpy gives you:
* the standard FP toolkit: `map`, `filter`, `reduce`
* useful transformations: len, `max`, `min`, `sum`
* other useful transformations: `enumerate`, `zip`, `groupby`, `accumulate`
* and even things you wouldn't think of as chainable: `reversed`, `sorted`, `join`, `print`
* plus even more!

Use the multi-line style (which is common in Java), so you can clearly read/understand each transformation and optionally add a comment/note to each line. After awhile, you notice how useful this style and think: "wow, I wish more things were like this!" It's just like bash's pipe `|` but in Python--probably useful for creating a *data pipeline*...  
Teaser: You mean like Spark 🤩 and Dask 😍? Fluentpy is not commonly used library 😢

In [4]:
import fluentpy as _ # install fluentpy if you aren't using mybinder.org

_("ba,dc").split(",").sorted(reverse=True).map(str.upper)._

('DC', 'BA')

In [5]:
(
    _(range(50))
    .map(_.each * 2) # fluentpy-specific syntax
    .map(lambda element: element * 2) # standard map + lambda
    .filter(_.each <= 170)
    .filter(lambda element: element % 20 == 0)
    .enumerate()
    .map(lambda element: "{}->{}".format(element[0], element[1]))
    .reduce(lambda x, y: ", ".join([x,y]))
    .split(", ")
    .print() # printi returns None
    .self # so use `self` to get the underlying elements
    .join(", ") # basically the same as the previous reduce
    .unwrap
)

['0->0', '1->20', '2->40', '3->60', '4->80', '5->100', '6->120', '7->140', '8->160']


'0->0, 1->20, 2->40, 3->60, 4->80, 5->100, 6->120, 7->140, 8->160'

### Dispatcher, Can You Get Me a Function?
`singledispatch()` (and `singledispatchmethod()`) is a way to write multiple versions of the same function--do something different for different argument types. (Multiple) dispatch is called multimethod or method overloading in other languages. Dispatch looks like Procedural Python because it is a switch statement for functions. Dispatch also looks like Object-Oriented Python because that's how you emulate method overloading in Python. Dispatch also looks like functional style because it is a decorator. But `singledispatch()` is not a pure function because it is mutating a dictionary under the hood for every `register()` call.

An example will help clarify how it works.

In [6]:
from functools import singledispatch

@singledispatch
def get_type(arg): # fallback/default function
    print("Unknown type")

@get_type.register(int) # specifically for int type
def _(arg):
    print("This is an integer type")

@get_type.register(str) # specifically for str type
def _(arg):
    print("This is a string type")

get_type(42) # calls the int type version of the function
get_type("42") # calls the string type version of the function
get_type( (42,) ) # no tuple type version of function, go to fallback/default option

This is an integer type
This is a string type
Unknown type


In [7]:
get_type.registry # this is a dictionary that gets updated every time you do get_type.register()

mappingproxy({object: <function __main__.get_type(arg)>,
              int: <function __main__._(arg)>,
              str: <function __main__._(arg)>})

The reason that Python calls it `singledispatch` is that you can only do it on the *first* argument's type. If you want to make the function differ based on the 2nd argument (and so on), then you can probably chain a bunch of `singledispatch`s together. The easier solution is to use a 3rd party, pip installable library that supports *multiple* dispatch (which there are many).

### Stack Skyscraper: Infinite Recursion Depth
If you combine FP with Procedural Python's exception handling, you can "beat" the recursion depth limit. In Python, you have a maximum stack depth to limit on how many nested function calls you can make. You can exceed this maximum limit by not going deep at all. What is this mysterious koan? I'll let the code do the talking.

In [8]:
class StackCheater(Exception):
    def __init__(self, *args, **kwargs):
        self.args = args
        self.kwargs = kwargs

def tail_recursive(f):
    def decorated(*args, **kwargs):
        while True:
            try:
                return f(*args, **kwargs)
            except StackCheater as sc:
                args = sc.args
                kwargs = sc.kwargs
                continue
    return decorated

@tail_recursive
def factorial(n, accumulator=1): # written in a tail recursive style with accumulator
    if n == 0:
        return accumulator
    else:
        raise StackCheater(n-1, accumulator=n*accumulator) # raise, not return

factorial(4000) # exceeds the default stack limit of 3000

1828801951514065013314743175573919044217377710730439219706452695420895979797317736485037028687048410733644304156928557175467246186154355733394261561795699671674528483159731749881876093748280498041957651294872061055892812978809780062059342953770532674062445388428509174395175674614444736237872246943619457592957990011421297336065899807397771469726120504866372593633749040406609796663717025402134880094428034228535594664968131626016345974380357717590339473317007684176477908216689118452932423003341414549780183259821851840655225709739253002458273898291910440678216870887149560350190586739996629879853487774792317919579141650440805487897477030865070712087883762498657607334044941485457836738330171570635819412740084985560408047330519683348240807942096427518753888911529665552239772392488715462481065978832100562055836960477865790477191838805431925151398195429674168844724618502125040222501011643301681858803669018017769146177971310430164039570827473470118677275696606461102365652876513873570419087620069

The trick I used is `try/except` to break out of any deep function calls. In fact, the call is only 2 levels deep, and it is technically still recursion. Deep recursion with a shallow call stack.

#### In Conclusion
The overall purpose of showing `fluentpy` (which is not commonly used), `singledispatch`, and infinite recursion isn't really to show fandangled ways to do things. The primary purpose is to show you how to think outside of the box. Don't be ~~dogmatic~~ a purist. There's a philosophical concept of thesis, antithesis, synthesis.
* *Thesis*: you come propose an idea.
* *Antithesis*: somebody else proposes an idea that seems to contradict yours.
* *Synthesis*: find a way that reconciles both ideas.

For example, somebody says Procedural Python is the best. Somebody else says Functional Python is the best. A pro says: I can make something even better by fusing the two styles. In reality, the best solution for your problem is a mixture of different styles. That's what makes Python fun! 😎

##  Extra Resources
* `itertools` library: has a bunch of tools that can be useful for FP: product, permutations, combinations, etc. Actually useful.
* `toolz` library: has even more useful tools under `functoolz` and `itertoolz` modules. Not commonly used.
* https://realpython.com/primer-on-python-decorators/: A long and exhaustive guide on decorators.
* https://www.destroyallsoftware.com/talks/boundaries and https://www.destroyallsoftware.com/screencasts/catalog/functional-core-imperative-shell: Gary Bernhardt (who created the technical talk series called "Destroy all Software") introduces the idea of functional core and imperative shell. The elementary piece of code should be pure functions, so it is easy to test and reason about. However, the larger "infrastructure" in a codebase needs to manipulates stdin/stdout, database, network, etc. Hence, the larger infrastructure can mutate things using imperative or OOP.
* David Beazley - Lambda Calculus from the Ground Up - PyCon 2019 (https://www.youtube.com/watch?v=pkCLMl0e_0k): If you want to ~~hurt yourself~~ learn more about Lambda Calculus, this video aims to teach you how to build everything from scatch starting from (very) elementary pieces: "No modules, no classes, no control flow, no data structures, and not even any primitives like integers or regular expressions. Just functions." It is also equally upfront with its disclaimer: "You will learn nothing practically useful in this tutorial."

## Concluding Remarks
If you come from OOP, then closures in FP are a poor man's objects. If you come from FP, then objects/classes in OOP are a poor man's closure. In a way, you can think of decorators as the FP version of inheritance in OOP: you extend/modify/override the behavior of the original function. The point is not to select a universal programming paradigm that you apply to everything. If you are given a hammer, then everything looks like a nail. Always keep in mind the Golden Rule of programming: **use the best tool for the problem.** Select between OOP and FP (and other styles) depending on how you think is best to attack the problem. Perhaps a less eloquent and more blunt way of saying this is: don't hurt yourself. Don't contort yourself with the mental gymnastics of fitting the problem into a fixed style; incorporate multiple styles to address the problem.

A rough rule of thumb is that data transformation *can be* more suited for FP design if  you don't need to remember (intermediate) state--in this case, it is more about the function purity. Things that need state or updates states (ike a UI design or database stuff) are more suited for OOP. OOP and FP are not polar opposites or mutually exclusive paradigms. A class's static method is like a compromise between FP and OOP. For example, a static method does not read or write the state of the instance (ie instance attributes) and is totally reliant on the inputted arguments. But a static method does not guarantee purity since it can still mutate a global variable. A static method uses OOP syntax because it is attached to a class.

Sometimes you can fuse OOP and FP to get the best of both worlds; a practical use case is high-performance frameworks (Apache Spark, Apache Beam, Dask [dataframe, bag, array], Ray) and graph-based engines (TensorFlow, Airflow, Prefect, Dask [delayed]).  For example since `map` and `filter` should use pure functions, they are easily parallelizable operations.

Highly parallelizable operations are ones where a transformation on 1 element is not dependent on another element and thus are called **Embarrassingly Parallel** problems. For example, word count is an example of a (mostly) embarrassingly parallel problem since you can count the words on different pages of a book using different processes (embarrassingly parallel) and then add up the intermediate results (not embarrassingly parallel) at the end to get the final answer. Guess what the next section is on? High-performance Python! ⚡